In [1]:
pip install finance-datareader

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
  # 데이터 불러우기
  df = pd.read_csv("/content/drive/MyDrive/학교/종목 추천/krx_stocks2000_2023(A).csv")

In [6]:
code_series = df['Code']
cleaned_codes = [code.replace('A', '') for code in code_series]
unique_cleaned_codes = list(set(cleaned_codes))
print(unique_cleaned_codes)

['344820', '052900', '005010', '294570', '035890', '090710', '016740', '108860', '057030', '217330', '078070', '257370', '014200', '013580', '277880', '086670', '026960', '334970', '000500', '084010', '004840', '039840', '036830', '022220', '002840', '025950', '031980', '120110', '143540', '000390', '091810', '039020', '096040', '230360', '002760', '000150', '009680', '317690', '332370', '194370', '038010', '001510', '013810', '002380', '002990', '002030', '003310', '078520', '238090', '019210', '052020', '003780', '018500', '153490', '215000', '036530', '332570', '049630', '065950', '007570', '049120', '043260', '279600', '057540', '038540', '234690', '033340', '094970', '056080', '300720', '241710', '054210', '065530', '078140', '286750', '091340', '002600', '036710', '001750', '034230', '123700', '060900', '041190', '021820', '072710', '339950', '365590', '208140', '089790', '214450', '009970', '029780', '383800', '007810', '000140', '048430', '210540', '079170', '052710', '004080',

In [ ]:
import pandas as pd
import FinanceDataReader as fdr

# 시작 날짜와 종료 날짜 설정
start_date = '2023-06-01'
end_date = '2023-09-03'

batch_size = 100  # 100개의 주가 코드씩 나눔

# 데이터프레임을 저장할 리스트
dfs = []

# 주가 코드를 배치로 나눠 데이터 가져오기
for i in range(0, len(unique_cleaned_codes), batch_size):
    batch_stock_codes = unique_cleaned_codes[i:i+batch_size]
    batch_dfs = []

    for stock_code in batch_stock_codes:
        stock_code_str = str(stock_code).zfill(6)
        df = fdr.DataReader(stock_code_str, start_date, end_date)
        # 주가 코드를 추가합니다.
        df['종목코드'] = stock_code_str

        # FinanceDataReader를 확장하여 종목명을 가져오는 함수 정의
        def get_company_name(stock_code):
            krx_listing = pd.read_csv("https://raw.githubusercontent.com/corazzon/finance-data-analysis/main/krx.csv")
            company_info = krx_listing[krx_listing['Symbol'] == stock_code]
            if not company_info.empty:
                return company_info.iloc[0]['Name']
            else:
                return "Unknown"  # 종목명을 찾을 수 없는 경우 "Unknown"으로 표시

        # 주가 코드로 종목명을 가져와서 추가
        company_name = get_company_name(stock_code_str)
        df['종목명'] = company_name

        # '일자' 열을 추가하고 날짜를 설정
        df['일자'] = df.index

        batch_dfs.append(df)

    # 각 배치 데이터프레임을 리스트에 추가
    dfs.extend(batch_dfs)

# 데이터프레임을 합칩니다.
combined_df = pd.concat(dfs, ignore_index=True)

# 합친 데이터프레임을 하나의 CSV 파일로 저장할 수도 있습니다.
combined_csv_filename = 'combined_stock_data.csv'
combined_df.to_csv(combined_csv_filename, index=False)

print(f"모든 주가 데이터를 {combined_csv_filename} 파일로 합쳤습니다.")

모든 주가 데이터를 combined_stock_data.csv 파일로 합쳤습니다.


In [ ]:
import pandas as pd

# CSV 파일 경로
file2 = '/content/drive/MyDrive/학교/종목 추천/combined_stock_data.csv'

# CSV 파일 로드
df = pd.read_csv(file2)

# 'Code' 열을 수정하여 A와 6자리 숫자 형식으로 만듭니다
df['종목코드'] = 'A' + df['종목코드'].astype(str).str.zfill(6)

# 수정된 데이터프레임을 저장
modified_csv_filename = '/content/drive/MyDrive/학교/종목 추천/modified_combined_stock_data.csv'
df.to_csv(modified_csv_filename, index=False)

print(f"'Code' 열의 값이 수정되어 {modified_csv_filename} 파일로 저장되었습니다.")

'Code' 열의 값이 수정되어 /content/drive/MyDrive/학교/종목 추천/modified_combined_stock_data.csv 파일로 저장되었습니다.


In [ ]:
import pandas as pd

# CSV 파일을 읽어 데이터프레임으로 변환
file_path = "/content/drive/MyDrive/학교/종목 추천/modified_combined_stock_data.csv"
df = pd.read_csv(file_path)

# 열 이름 변경
df.rename(columns={'Open': '시가', 'High': '고가', 'Low': '저가', 'Close': '종가', 'Volume': '거래량'}, inplace=True)

# Change 열 삭제
df.drop('Change', axis=1, inplace=True)

print(df)

           시가     고가     저가     종가      거래량     종목코드     종목명          일자
0       12000  12000  11670  11890    43458  A317870  엔바이오니아  2023-06-01
1       11890  12890  11800  12800   142035  A317870  엔바이오니아  2023-06-02
2       13050  13050  12660  12770    45274  A317870  엔바이오니아  2023-06-05
3       12650  12710  12400  12600    57474  A317870  엔바이오니아  2023-06-07
4       12510  13350  12430  12860    60688  A317870  엔바이오니아  2023-06-08
...       ...    ...    ...    ...      ...      ...     ...         ...
129942  80800  85700  80000  80400  1362120  A000250   삼천당제약  2023-08-28
129943  85000  91300  83600  86200  2423301  A000250   삼천당제약  2023-08-29
129944  85600  88800  85400  87400   671591  A000250   삼천당제약  2023-08-30
129945  86600  90500  84800  87900   745401  A000250   삼천당제약  2023-08-31
129946  86900  91300  86000  88600   554301  A000250   삼천당제약  2023-09-01

[129947 rows x 8 columns]


In [ ]:
output_file_path = "/content/drive/MyDrive/학교/종목 추천/modified_stock_data.csv"
df.to_csv(output_file_path, index=False)  # index=False를 지정하여 인덱스 열을 저장하지 않도록 합니다

In [ ]:
import pandas as pd

# CSV 파일 경로
file1 = '/content/drive/MyDrive/학교/종목 추천/modified_stock_data.csv'
file2 = '/content/drive/MyDrive/학교/종목 추천/train.csv'

# CSV 파일 로드
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# 두 데이터프레임을 합침
combined_df = pd.concat([df1, df2], ignore_index=True)

# 합친 데이터프레임을 하나의 CSV 파일로 저장
combined_csv_filename = '/content/drive/MyDrive/학교/종목 추천/train_data.csv'
combined_df.to_csv(combined_csv_filename, index=False)

print(f"두 개의 CSV 파일을 {combined_csv_filename} 파일로 합쳤습니다.")

두 개의 CSV 파일을 /content/drive/MyDrive/학교/종목 추천/train_data.csv 파일로 합쳤습니다.


In [ ]:
import pandas as pd
import FinanceDataReader as fdr

# 시작 날짜와 종료 날짜 설정
start_date = '2023-09-04'
end_date = '2023-11-01'

batch_size = 100  # 100개의 주가 코드씩 나눔

# 데이터프레임을 저장할 리스트
dfs = []

# 주가 코드를 배치로 나눠 데이터 가져오기
for i in range(0, len(unique_cleaned_codes), batch_size):
    batch_stock_codes = unique_cleaned_codes[i:i+batch_size]
    batch_dfs = []

    for stock_code in batch_stock_codes:
        stock_code_str = str(stock_code).zfill(6)
        df = fdr.DataReader(stock_code_str, start_date, end_date)
        # 주가 코드를 추가합니다.
        df['종목코드'] = stock_code_str

        # FinanceDataReader를 확장하여 종목명을 가져오는 함수 정의
        def get_company_name(stock_code):
            krx_listing = pd.read_csv("https://raw.githubusercontent.com/corazzon/finance-data-analysis/main/krx.csv")
            company_info = krx_listing[krx_listing['Symbol'] == stock_code]
            if not company_info.empty:
                return company_info.iloc[0]['Name']
            else:
                return "Unknown"  # 종목명을 찾을 수 없는 경우 "Unknown"으로 표시

        # 주가 코드로 종목명을 가져와서 추가
        company_name = get_company_name(stock_code_str)
        df['종목명'] = company_name

        # '일자' 열을 추가하고 날짜를 설정
        df['일자'] = df.index

        batch_dfs.append(df)

    # 각 배치 데이터프레임을 리스트에 추가
    dfs.extend(batch_dfs)

# 데이터프레임을 합칩니다.
combined_df = pd.concat(dfs, ignore_index=True)

# 합친 데이터프레임을 하나의 CSV 파일로 저장할 수도 있습니다.
combined_csv_filename = 'train_additional.csv'
combined_df.to_csv(combined_csv_filename, index=False)

print(f"모든 주가 데이터를 {combined_csv_filename} 파일로 합쳤습니다.")

모든 주가 데이터를 train_additional.csv 파일로 합쳤습니다.


In [7]:
import pandas as pd
import FinanceDataReader as fdr

# 시작 날짜와 종료 날짜 설정
start_date = '2023-11-01'
end_date = '2023-12-28'

batch_size = 100  # 100개의 주가 코드씩 나눔

# 데이터프레임을 저장할 리스트
dfs = []

# 주가 코드를 배치로 나눠 데이터 가져오기
for i in range(0, len(unique_cleaned_codes), batch_size):
    batch_stock_codes = unique_cleaned_codes[i:i+batch_size]
    batch_dfs = []

    for stock_code in batch_stock_codes:
        stock_code_str = str(stock_code).zfill(6)
        df = fdr.DataReader(stock_code_str, start_date, end_date)
        # 주가 코드를 추가합니다.
        df['종목코드'] = stock_code_str

        # FinanceDataReader를 확장하여 종목명을 가져오는 함수 정의
        def get_company_name(stock_code):
            krx_listing = pd.read_csv("https://raw.githubusercontent.com/corazzon/finance-data-analysis/main/krx.csv")
            company_info = krx_listing[krx_listing['Symbol'] == stock_code]
            if not company_info.empty:
                return company_info.iloc[0]['Name']
            else:
                return "Unknown"  # 종목명을 찾을 수 없는 경우 "Unknown"으로 표시

        # 주가 코드로 종목명을 가져와서 추가
        company_name = get_company_name(stock_code_str)
        df['종목명'] = company_name

        # '일자' 열을 추가하고 날짜를 설정
        df['일자'] = df.index

        # 'Change' 열을 추가
        df['변동률'] = (df['Close'] - df['Open']) / df['Open'] * 100

        batch_dfs.append(df)

    # 각 배치 데이터프레임을 리스트에 추가
    dfs.extend(batch_dfs)

# 데이터프레임을 합칩니다.
combined_df = pd.concat(dfs, ignore_index=True)

# 합친 데이터프레임을 하나의 CSV 파일로 저장할 수도 있습니다.
combined_csv_filename = 'combined_stock_data3.csv'
combined_df.to_csv(combined_csv_filename, index=False)

print(f"모든 주가 데이터를 {combined_csv_filename} 파일로 합쳤습니다.")

KeyboardInterrupt: ignored

In [ ]:
combined_df = pd.read_csv("/content/drive/MyDrive/학교/종목 추천/combined_stock_data2.csv")

In [ ]:
# 데이터 열의 이름을 변경
combined_df.rename(columns={
    '일자': 'Date',
    '종목코드': 'Code'
}, inplace=True)
# "종목명" 열 삭제
# "종목명" 열 삭제
combined_df.drop(columns=['종목명'], inplace=True)
# 열이 삭제된 데이터프레임 확인
print(combined_df.head())

    Open   High    Low  Close  Volume    Change    Code        Date
0  25150  27050  23800  26600  317491  0.031008  214260  2023-07-28
1  27200  27850  26350  26950  199699  0.013158  214260  2023-07-31
2  26700  28700  26200  28400  379073  0.053803  214260  2023-08-01
3  27850  28300  25550  27100  245957 -0.045775  214260  2023-08-02
4  26800  27850  25700  27050  122243 -0.001845  214260  2023-08-03


In [ ]:
combined_df['Code'] = 'A' + combined_df['Code'].astype(str).str.zfill(6)

In [ ]:
print(combined_df.head())

    Open   High    Low  Close  Volume    Change     Code        Date
0  25150  27050  23800  26600  317491  0.031008  A214260  2023-07-28
1  27200  27850  26350  26950  199699  0.013158  A214260  2023-07-31
2  26700  28700  26200  28400  379073  0.053803  A214260  2023-08-01
3  27850  28300  25550  27100  245957 -0.045775  A214260  2023-08-02
4  26800  27850  25700  27050  122243 -0.001845  A214260  2023-08-03


In [ ]:
combined_csv_filename = 'combined_df.csv'
combined_df.to_csv(combined_csv_filename, index=False)

In [ ]:

import pandas as pd

# CSV 파일 경로
file1 = '/content/drive/MyDrive/학교/종목 추천/krx_stocks2000_2023(A).csv'
file2 = '/content/drive/MyDrive/학교/종목 추천/combined_df.csv'

# CSV 파일 로드
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# 두 데이터프레임을 합침
combined_df = pd.concat([df1, df2], ignore_index=True)

# 합친 데이터프레임을 하나의 CSV 파일로 저장
combined_csv_filename = '/content/drive/MyDrive/학교/종목 추천/good.csv'
combined_df.to_csv(combined_csv_filename, index=False)

print(f"두 개의 CSV 파일을 {combined_csv_filename} 파일로 합쳤습니다.")

두 개의 CSV 파일을 /content/drive/MyDrive/학교/종목 추천/good.csv 파일로 합쳤습니다.
